In [3]:
import nltk # Natural Language Toolkit
nltk.download('stopwords') #предлоги, союзы

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
# dataset - цель научить машину идентифицировать авторов по их стилю 
import numpy as np
import pandas as pd 

import sklearn
from sklearn.pipeline import FeatureUnion # delete предлоги, союзы
from sklearn.model_selection import train_test_split

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [53]:
df=pd.read_csv(r'pipeline-author-identification.csv',header=0)
df=df.dropna(axis = 0) #Удаление пропущенных данных
print(df.shape) #(19579 - строк, 9 - столбцов)
df.set_index('id',inplace= True)
df.head()

(19579, 9)


,text,author,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,,,
id26305,"This process, however, afforded me no means of...",EAP,this process however afforded me no means of a...,224,41,21,6.380952,4
id17569,It never once occurred to me that the fumbling...,HPL,it never once occurred to me that the fumbling...,70,14,6,6.166667,0
id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...,195,36,19,5.947368,4
id27763,How lovely is spring As we looked from Windsor...,MWS,how lovely is spring as we looked from windsor...,202,34,21,6.476190,3
id12958,"Finding nothing else, not even gold, the Super...",HPL,finding nothing else not even gold the superin...,170,27,16,7.187500,2


In [54]:
features = [c for c in df.columns.values if c not in ['id', 'text', 'author']] #not numeric values
numeric_features = [c for c in df.columns.values if c not in ['id', 'text', 'author', 'processed']]

target = 'author'

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size = 0.33, random_state=42)
X_train.head()

,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,
id19417,this panorama is indeed glorious and i should ...,91,18,6,6.666667,1
id09522,there was a simple natural earnestness about h...,240,44,18,6.277778,4
id22732,who are you pray that i duc de lomelette princ...,387,74,38,5.552632,9
id10351,he had gone in the carriage to the nearest tow...,118,24,11,5.363636,0
id24580,there is no method in their proceedings beyond...,71,13,5,7.000000,1


In [58]:
class TextSelector(BaseEstimator, TransformerMixin): #text feature
    def __init__(self,key):
        self.key = key
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        return X[self.key]

In [59]:
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        
    def fit (self, X, y = None):
        return self
    
    def transform (self, X):
        return X[[self.key]]

In [60]:
text = Pipeline ([
    ('selector', TextSelector(key='processed')),
    ('tfidf', TfidfVectorizer(stop_words = 'english'))
])
text.fit_transform(X_train)

<13117x21516 sparse matrix of type '<class 'numpy.float64'>'
	with 148061 stored elements in Compressed Sparse Row format>

In [62]:
length = Pipeline([
    ('selector', NumberSelector(key='length')),
    ('standard', StandardScaler())
])
length.fit_transform(X_train)

array([[-0.50769254],
       [ 0.88000324],
       [ 2.24907223],
       ...,
       [-0.46112557],
       [-0.14447015],
       [-0.39593181]])

In [69]:
words = Pipeline([
    ('selector', NumberSelector(key='words')),
    ('standard', StandardScaler())
])

words_not_stopword = Pipeline([
    ('selector', NumberSelector(key='words_not_stopword')),
    ('standard', StandardScaler())
])

avg_word_length= Pipeline([
    ('selector', NumberSelector(key='avg_word_length')),
    ('standard', StandardScaler())
])

commas= Pipeline([
    ('selector', NumberSelector(key='commas')),
    ('standard', StandardScaler())
])

feats = FeatureUnion([
    ('text', text),
    ('length', length),
    ('words', words),
    ('words_not_stopword', words_not_stopword),
    ('avg_word_length', avg_word_length),
    ('commas', commas)
])


In [70]:
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

<13117x21521 sparse matrix of type '<class 'numpy.float64'>'
	with 213646 stored elements in Compressed Sparse Row format>

In [71]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state = 42))
])

pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
np.mean(preds == y_test)

0.6792014856081708